
**Yule-Walker Equations**

The Yule-Walker can be used to find the optimal parameters for an AR(p) model.

$$ x_t = \phi_1 x_{t-1} + \phi_2 x_{t-2} + ... + \phi_p x_{t-p} + w_t$$

The equations arises as the solution of estimating the optimal coeffecients for Best Linear Prediction.
Another way to derive the equations are to multiply by \\(x_{t-h}\\) and take the expectation of each sides of the AR(p) process.

$$ E[x_t x_{t-h}] = \phi_1 E[x_{t-1} x_{t-h}] + \phi_2 E[x_{t-2} x_{t-h}] + ... + \phi_p E[x_{t-p} x_{t-h}] + E[w_t x_{t-h}]$$

Since \\(E[x_tx_{t-h}]=\gamma(h)\\) and \\(w_t\\) does not depend on any prior \\(x_{t-h}\\) such that \\(E[w_t x_{t-h}]=0\\), we can simplify to.
 
$$ \gamma(h) = \phi_1 \gamma(h-1) + \phi_2 \gamma(h-2) + ... + \phi_p \gamma(h-p) $$

This is known as the Yule-Walker recursion. For \\(h = 1,2,..,p\\) this makes up the Yule-Walker equations.

The process noise variance can also be estimated using the Yule-Walker equations by considering the special case \\(h=0\\)

$$ E[x_t x_t] = \phi_1 E[x_{t-1} x_t] + \phi_2 E[x_{t-2} x_t] + ... + \phi_p E[x_{t-p} x_t] + E[w_t x_t]$$

$$ \gamma(0) = \phi_1 \gamma(1) + \phi_2 \gamma(2) + ... + \phi_p \gamma(p) + E[w_t x_t]$$

We can expand \\(E[w_t x_t]\\) to

$$ E[w_t x_t] = E[w_t (\phi_1 x_{t-1} + \phi_2 x_{t-2} + ... + \phi_p x_{t-p} + w_t)] = E[ \phi_1 x_{t-1} w_t + \phi_2 x_{t-2} w_t + ... + \phi_p x_{t-p} w_t + w_t w_t)] = E[w_t w_t]=\sigma_w^2$$

Since again \\(w_t\\) does not depend on any prior \\(x_{t-h}\\)

$$ \gamma(0) = \phi_1 \gamma(1) + \phi_2 \gamma(2) + ... + \phi_p \gamma(p) + \sigma_w^2$$

Solving for \\(\sigma_w^2\\) yields

$$ \sigma_w^2 = \gamma(0) - \phi_1 \gamma(1) + ... + \phi_p \gamma(p) $$

All of this can be written in matrix notation as 

$$ \mathbf{\Gamma_p} \mathbf{\phi} = \mathbf{\gamma_p} $$

$$ \sigma_w^2 = \gamma(0) - \mathbf{\gamma_p}^T\mathbf{\Gamma_p}^{-1} \gamma_p = \gamma(0) - \mathbf{\phi}^T \mathbf{\gamma}_p $$

The Yule-Walker can be factored by \\(\gamma(0)\\) to depend on the autocorrelation instead of the autocovariance.

$$ \widehat{\mathbf{\phi}} = \widehat{\mathbf{\normalsize{R}}}_p^{-1} \widehat{\mathbf{\rho}}_p $$

$$ \sigma_w^2 = \widehat{\gamma}(0)(1 - \widehat{\mathbf{\rho}}_p^T \widehat{\mathbf{\normalsize{R}}}_p^{-1} \widehat{\rho}_p) $$


In [0]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import scipy.stats as stats
import matplotlib.pyplot as plt

# Define autocovariance
def autocovariance(x, lag):
    n = len(x)
    mean_x = np.mean(x)
    return np.sum((x[:n - lag] - mean_x) * (x[lag:] - mean_x)) / n

## Define autocovariance matrix
def autocov_matrix(x, max_lag):
    return np.array([[autocovariance(x, abs(i - j)) for j in range(max_lag + 1)] for i in range(max_lag + 1)])

def gen_ar_data(num_samples, sigma, mu, phi):
    w = sigma * np.random.randn(num_samples) + mu
    x = np.zeros(num_samples)   
    
    # Generate data for the model
    # x[n] = phi_1 * x[n-1] + phi_2 * x[n-2] + .. + phi_k * x[n-k] + w[n]
    x = signal.lfilter(b=[1], a=[1] + [-phi_ for phi_ in phi], x=w)
    return x, w

def calc_ar_coefs(x, num_model_coefs):
    cov = autocov_matrix(x, num_model_coefs)
    # Solve the equation Ax=b to find the coefficients
    Gamma = cov[-num_model_coefs:, -num_model_coefs:]
    gamma = cov[1:, 0]
    coefs = np.linalg.inv(Gamma) @ gamma
    return coefs, cov, Gamma, gamma


In [0]:
# Example on generating AR(2) process data and estimating the parameters

N = 1000
mu, sigma = 0, 0.1
phi = [1.5, -0.75]
# We know the number of model coeficients
num_model_coefs = len(phi)

x, w = gen_ar_data(N, sigma, mu, phi)

coefs, cov, Gamma, gamma = calc_ar_coefs(x, num_model_coefs)

print(f"Phi: {phi}, estimated phi: {coefs}")


**Autocovariance matrix of an AR(2) process**

Using the Yule-Walker equations and an AR(2) process, we can find the autocovariance matrix.

The Yule-Walker equations for an AR(2) process are

$$ \gamma(0) = \phi_1 \gamma(1) + \phi_2 \gamma(2) + \sigma_w^2 $$

$$ \gamma(1) = \phi_1 \gamma(0) + \phi_2 \gamma(1) $$

$$ \gamma(2) = \phi_1 \gamma(1) + \phi_2 \gamma(0) $$

This can be rewritten to

$$ \gamma(0) - \phi_1 \gamma(1) - \phi_2 \gamma(2) = \sigma_w^2 $$

$$ - \phi_1 \gamma(0) + \gamma(1) (1-\phi_2) = 0 $$

$$ - \phi_2 \gamma(0) - \phi_1 \gamma(1) + \gamma(2) = 0 $$

In matrix notation this becomes

$$ \mathbf{\Phi} \mathbf{\gamma} = [\sigma^2, 0, 0]^T$$

Where \\(\mathbf{\gamma}=[\gamma(0), \gamma(1), \gamma(2)]^T\\) and \\(\mathbf{\Phi} =[[1, -\phi_1, -phi_2],[-phi_1, (1-phi_2), 0],[-phi_2, -phi_1, 1]]\\)

In [0]:
## Example on how to calculate the covariance matrix given an AR(2) process
# It can be seen that as N increases, the esimated covariance matrix converges to the true covariance matrix

Ns = [2**i for i in range(6, 16)]
num_tests = 100

# Calulate true covariance matrix
Phi = np.array([[1, -phi[0], -phi[1]],
                [-phi[0], (1-phi[1]), 0],
                [-phi[1], -phi[0], 1]])

b = [sigma**2, 0, 0]

gamma_true = np.linalg.inv(Phi) @ b
Gamma_true = np.array([[gamma_true[0], gamma_true[1], gamma_true[2]],
                       [gamma_true[1], gamma_true[0], gamma_true[1]],
                       [gamma_true[2], gamma_true[1], gamma_true[0]]])

covariance_errors = np.zeros(len(Ns))

for n, N in enumerate(Ns):
    for i in range(num_tests):
        x, w = gen_ar_data(N, sigma, mu, phi)

        coefs, cov, Gamma, gamma = calc_ar_coefs(x, num_model_coefs)
        error = np.sum(np.abs(cov - Gamma_true))
        covariance_errors[n] += error

covariance_errors = covariance_errors / num_tests

plt.figure()
plt.plot(Ns, covariance_errors, label="Covariance matrix error")
plt.legend()
plt.yscale("log")
plt.xlabel("Samples")
plt.ylabel("Error")


**Property - Large Sample Error of Yule-Walker Estimators**

The asymptotic behaviour as \\(N \rightarrow \infin\\) of the Yule-Walker Estimators in an causal AR(p) is as following

$$ \sqrt(n) (\mathbf{\widehat{\phi}} - \mathbf{\phi}) \xrightarrow{d} \mathcal{N}(\mu, \sigma_w^2 \Gamma_p^{-1})$$

Here \\(\Gamma_p\\) is the autocovariance toeplitx matrix with values from \\(\gamma(0)\\) to \\(\gamma(p)\\).

As N gets larger, the error will converge in distribution towards being normally distributed with \\(\mu=0\\) and \\(\sigma^2 = \sigma_w^2 \Gamma_p^{-1}\\)


In [0]:
# In this example it is see n how the error distribution of parameters
# converges towards the theoretical distribution 

num_samples_test = [2**i for i in range(10, 20)]
num_test = 100

estimator_errors = np.zeros((len(num_samples_test), num_test))
estimated_noise_variance = np.zeros((len(num_samples_test), num_test))
estimator_errors_var_theoretical = np.zeros(len(num_samples_test))


for i, num_samples in enumerate(num_samples_test):
    for j in range(num_test):

        x, w = gen_ar_data(num_samples, sigma, mu, phi)

        coefs, cov, Gamma, gamma = calc_ar_coefs(x, num_model_coefs)
        estimator_errors[i, j] = (coefs[0] - phi[0])
        estimated_noise_variance[i, j] = cov[0, 0] - gamma.T @ np.linalg.inv(Gamma) @ gamma

    phi_cov = (sigma**2)/num_samples * np.linalg.inv(Gamma_true[:2, :2])
    
    estimator_errors_var_theoretical[i] = phi_cov[0, 0]


plt.figure()
plt.plot(num_samples_test, estimator_errors_var_theoretical, label="Estimator error variance theoretical")
plt.plot(num_samples_test, np.var(estimator_errors, axis=1, ddof=1), label="Estimator error variance empirical")
plt.legend()
plt.yscale("log")

plt.figure()
plt.plot(num_samples_test, np.zeros_like(num_samples_test), label="Estimator error mean theoretical")
plt.plot(num_samples_test, np.mean(estimator_errors, axis=1), label="Estimator error mean empirical")
plt.legend()

plt.figure()
plt.plot(num_samples_test, np.mean(estimated_noise_variance, axis=1), label="Estimated noise variance")
plt.plot(num_samples_test, np.ones(len(num_samples_test))*sigma**2, label="True noise variance")
plt.legend()
plt.xlabel("Samples")
plt.ylabel("Noise variance")
plt.yscale("log")


**Maximum likelihood estimation of parameters**

Another way of estimating AR parameters is using a maximum likelihood method.

Consider the AR(1) process

$$ x_t = \phi x_{t-1} + w_t $$

Where \\(w_t \sim \mathcal{N}(0, \sigma_w^2)\\) and \\(x_t \sim \mathcal{N}(0, \frac{\sigma_w^2}{1-\phi^2})\\)

The joint likelihood, for \\(x_1, x_2,..,x_T\\) is

$$ L(\phi, \sigma_w^2) = f(x_1, x_2,..,x_T |\phi, \sigma_w^2) $$

The likelihood is the measure of how likely we are to observe \\([x_1, x_2,..,x_T]\\) given the model parameters \\([\phi, \sigma_w^2]\\)

Since \\(x_1\\) does not depend on any future values of \\(x_t\\) we can factor it out.
For convenience, the model parameters are left out.

$$ f(x_1, x_2,..,x_T) = f(x_1)f(x_2,..,x_T) $$

Using the chain rule of probability \\(P(A,B) = P(A|B)P(B)\\) we can split up the likelihood function. We can factor out \\(f(x_2)\\) if we condition on already knowing \\(x_1\\). 

$$ f(x_1, x_2,..,x_T) = f(x_1)f(x_2|x_1)f(x_3,..,x_T) $$

We can continue to do this indefinitely which will give us

$$ f(x_1, x_2,..,x_T) = f(x_1) \prod_{t=2}^T f(x_k|x_{t-1}) $$

We consider \\(x_1\\) is known, so \\(f(x_1)=1\\) and the likelihood function becomes

$$ L(\phi, \sigma_w^2) = \prod_{t=2}^T f(x_k|x_{t-1};\phi, \sigma_w^2) $$

Here \\(x_t|x_{t-1} \sim \mathcal{N}(\phi x_{t-1}, \sigma_w^2)\\) since \\(x_t\\) is made up of the known quantity \\(\phi x_{t-1}\\) and the random variable \\(w_t\\)

$$ L(\phi, \sigma_w^2) = \prod_{t=2}^T \frac{1}{\sqrt{2 \pi \sigma^2}} e^{-\frac{(x_t-\phi x_{t-1})^2}{2 \sigma_w^2}} $$

We find the log likelihood by taking the logarithm to both sides

$$ \ell(\phi, \sigma_w^2) = - \frac{T-1}{2}log(2 \pi \sigma_w^2) - \frac{1}{2 \pi \sigma_w^2} \sum_{t=2}^T(x_t - \phi x_{t-1})^2 $$

We want to maximize the likelihood and hence maximize the log likelihood. This is done with respect to \\( \phi \\). To maximize the log likelihood, we minimize the expression \\(S(\phi)\sum_{t=2}^T(x_t - \phi x_{t-1})^2\\)

This can either be done by taking the derivative with respect to \\(\phi\\), setting it equal to 0 and solving. However, we recognize the term as the sum of least squares, which means that when we condition on one step ahead, the MLE is also the least squates estimator.

In [0]:
# Example showing how to estimate the AR process parameters using the MLE

N = 50
mu, sigma = 0, 0.1
phi = 0.5

w = sigma * np.random.randn(N) + mu
x = np.zeros(N)
x_hat = np.zeros(N)   

x[0] = w[0]
for i in range(1, len(x)):
    x[i] = phi * x[i-1] + w[i]

# Construct y
y = x[1:]
v = x[:-1]

# Solve the OLS linear regression problem
phi_est =  np.dot(v, y) / (np.dot(v,v))

print(f"Coefficient estimated with MLE: {phi_est}")

x_hat[0] = w[0]
for i in range(1, len(x_hat)):
    x_hat[i] = phi_est * x_hat[i-1] + w[i]

plt.figure()
plt.plot(x, label="True signal")
plt.plot(x_hat, label="Estimated signal")
plt.xlabel("Samples")
plt.ylabel("Amplitude")
plt.legend()
plt.title("Estimated signal vs true signal")


** Estimation of both AR and MA parameters **

Given an ARMA(p,q) model

$$ x_t = \sum_{i=1}^p \phi_i x_{t-i} + w_t + \sum_{j=1}^q \theta_j w_{t-j} $$

We see that for estimating the AR paramters \\(\phi\\) we can apply linear regression to find the paramters \\(\phi\\) that give the best linear combination of \\(x_{t-1},x_{t-2},..,x_{t-p}\\) that equals \\(x_t\\).

However, for the MA part of the process, we cannot directly observe the \\(w_t\\) terms and since they are dependent on the model paramters. Hence, to estimate the model parameters, we need to estimate the errors, which in turn depends in the model parameters. This is a non-linear problem without any closed form solution and we'll need numerical methods to solve it.

**Guass-Newton method**

